In [1]:
import itertools
import numpy as np
from math import factorial

def _embed(x, order=3, delay=1):
    N = len(x)
    Y = np.zeros((order, N - (order - 1) * delay))
    for i in range(order):
        Y[i] = x[i * delay : i * delay + Y.shape[1]]
    return Y.T

def permutation_entropy(x, order=3, delay=1, normalize=False):
    """Permutation Entropy (Bandt and Pompe 2002)

    Parameters
    ----------
    x : list or np.array
        Time series
    order : int
        Order of permutation entropy
    delay : int
        Time delay
    normalize : bool
        If True, divide by log2(factorial(order)) to normalize the entropy
        between 0 and 1. Otherwise, return the permutation entropy in bit.

    Returns
    -------
    pe : float
        Permutation Entropy

    References
    ----------
    .. [1] Massimiliano Zanin et al. Permutation Entropy and Its Main
        Biomedical and Econophysics Applications: A Review.
        http://www.mdpi.com/1099-4300/14/8/1553/pdf
    .. [2] Christoph Bandt and Bernd Pompe. Permutation entropy — a natural
        complexity measure for time series.
        http://stubber.math-inf.uni-greifswald.de/pub/full/prep/2001/11.pdf

    Examples
    --------
    1. Permutation entropy with order 2
        >>> x = [4, 7, 9, 10, 6, 11, 3]
        >>> # Return a value between 0 and log2(factorial(order))
        >>> print(permutation_entropy(x, order=2))
            0.918
    2. Normalized permutation entropy with order 3
        >>> x = [4, 7, 9, 10, 6, 11, 3]
        >>> # Return a value comprised between 0 and 1.
        >>> print(permutation_entropy(x, order=3, normalize=True))
            0.589
    """
    x = np.array(x)
    ran_order = range(order)
    hashmult = np.power(order, ran_order)
    # Embed x and sort the order of permutations
    sorted_idx = _embed(x, order=order, delay=delay).argsort(kind='quicksort')
    # Associate unique integer to each permutations
    hashval = (np.multiply(sorted_idx, hashmult)).sum(1)
    # Return the counts
    _, c = np.unique(hashval, return_counts=True)
    # Use np.true_divide for Python 2 compatibility
    p = np.true_divide(c, c.sum())
    pe = -np.multiply(p, np.log2(p)).sum()
    if normalize:
        pe /= np.log2(factorial(order))
    return pe

In [5]:
x = [4, 7, 9, 10, 6, 11, 3]
print(permutation_entropy(x, order=2))
print(permutation_entropy(x, order=3, normalize=False))

0.9182958340544896
1.5219280948873621


In [3]:
def hash_term(perm):
    """Associate unique integer to a permutation.
    """
    deg = len(perm)
    return sum([perm[k]*deg**k for k in range(deg)])


def permutation_entropy_orig(x, order=3, delay=1, normalize=False):
    """Permutation Entropy (Bandt and Pompe 2002)

    Parameters
    ----------
    x : list or np.array
        Time series
    order : int
        Order of permutation entropy
    delay : int
        Time delay
    normalize : bool
        If True, divide by log2(factorial(order)) to normalize the entropy
        between 0 and 1. Otherwise, return the permutation entropy in bit.

    Returns
    -------
    pe : float
        Permutation Entropy

    References
    ----------
    .. [1] Massimiliano Zanin et al. Permutation Entropy and Its Main
        Biomedical and Econophysics Applications: A Review.
        http://www.mdpi.com/1099-4300/14/8/1553/pdf

    .. [2] Christoph Bandt and Bernd Pompe. Permutation entropy — a natural
        complexity measure for time series.
        http://stubber.math-inf.uni-greifswald.de/pub/full/prep/2001/11.pdf

    Examples
    --------
    1. Permutation entropy with order 2

        >>> x = [4, 7, 9, 10, 6, 11, 3]
        >>> # Return a value between 0 and log2(factorial(order))
        >>> print(permutation_entropy(x, order=2))
            0.918

    2. Normalized permutation entropy with order 3

        >>> x = [4, 7, 9, 10, 6, 11, 3]
        >>> # Return a value comprised between 0 and 1.
        >>> print(permutation_entropy(x, order=3, normalize=True))
            0.589
    """
    x = np.array(x)
    permutations = list(itertools.permutations(range(order)))
    hashlist = [hash_term(perm) for perm in permutations]
    c = np.zeros(len(permutations), dtype=int)
    ran = np.arange(len(x) - delay * (order - 1))
    step = ran + order * delay

    for i in ran:
        sorted_idx = x[i:step[i]:delay].argsort(kind='quicksort')
        c[np.nonzero(hashlist == hash_term(sorted_idx))[0][0]] += 1

    c = c[np.nonzero(c)]
    # Use np.true_divide for Python 2 compatibility
    p = np.true_divide(c, c.sum())
    pe = -np.multiply(p, np.log2(p)).sum()
    if normalize:
        pe /= np.log2(factorial(order))
    return pe

In [4]:
x = np.random.rand(30000)
# x = [4, 7, 9, 10, 6, 11, 3]

print(permutation_entropy(x, order=2, normalize=True))
print(permutation_entropy_orig(x, order=2, normalize=True))

%timeit permutation_entropy(x, order=3)
%timeit permutation_entropy_orig(x, order=3)

0.9999792229050628
0.9999792229050628
2.14 ms ± 57.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
256 ms ± 2.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
